In [15]:
import pyArango.connection as arc
import numpy as np
import json as js

In [ ]:
"""
componer la url con la ip del server en puerto 8529
poner el usario y contraseña que les di
"""
conn = arc.Connection(arangoURL="" ,username="", password="")

Creamos una base de datos 
Por default Arango intenta establecer conexion a http://127.0.0.1:8529. Si tienes alojado ArangoDB en un server diferente necesitas poner esa opción cuando crees una instancia de la clase Connection .  

En caso de que la base de datos ya exista para abrirla, suponiendo que se llama Facultad se hace de la siguiente manera

In [40]:
#creamos la base de datos
#db = conn.createDatabase(name="fciencias") #ustedes no tienen este permiso la base ya esta 
#creada y ya se les asigno
#cargamos la base por default del sistema
db = conn["fciencias"] #cargar esta base de datos

Los **documentos** que usted almacena en ArangoDB siguen de cerca el formato JSON, sin embargo se almacenan como un formato binario llamado VelocyPack, en MongoDB el formato binario  es BJSON . Un documento contiene cero o mas atributos y cada uno de esos atributos tienen un valor. Un valor puede ser un tipo átomico es decir un número, un string, boleano o nulo. O un tipo compuesto
como lo es un array o documento embebido / objetos. 
Los arrays y los sub-bjetos pueden contener todos estos tipos, lo que significa que las estructuras de datos anidadas arbitrariamente se pueden representar en un solo documento.

Los documentos se agrupan en **Colecciones**. Las colecciones continen cero o más documentos. Si estas familiarizado con (RDBMS), entónces asociaras las colecciones con las tablas y las filas con los documentos. La gran diferencia es que las (RDBMS) tradicionales se tienen que definir columnas antes de empezar a almacenar registros en una tabla, tales definiciones son los esquemas (una agrupación lógica para los artefactos de base de datos). **ArangoDB** es un schema-less, esto quiere decir que no necesitas definir que atributos debe tener un documento. 

Hay 2 tipos de colecciones: colecciones de documentos (también conocidos como una colecciones de vertices en el contexto de gráficas) así como colecciones de aristas. Las colecciones de aristas también almacenan documentos pero incluyen el atributo especial (_from y _to ) los cuales se usan para crear relaciones entre documentos. Dos documentos (vertices) almacenados en una colección de documentos son vinculados por un documento (arista) almacenado en una colección. Por eso **ArangoBD** esta orientado a grafos. Sigue el concepto matemático de grafo dirigido. y etiquetado, excepto que en los nodos no sólo tienen eqiquetas si no documentos. 
**:O :O**

In [20]:
#db.createCollection(name="Estudiantes")

In [21]:
#cargamos la coleccion 
coleccionEstudiantes=db['Estudiantes']
coleccionEstudiantes

ArangoDB collection name: Estudiantes, id: 1869, type: document, status: loaded

Ahora vamos a crear un documento

In [24]:
doc0=coleccionEstudiantes.createDocument()
doc0["nombre"]="Juanito Perez jolote"
doc0.save() #guardamos el documento

In [25]:
doc1=coleccionEstudiantes.createDocument()
doc1["nombre"]="juan2"
doc1["apellido"]="Gonzalez"
doc1["Calif"]=[10,2,9,7,8]
doc1._key = "juan"
doc1.save()

### AQL
Arango DB cuenta con un lenguaje de consulta (AQL=ArangoDB Query Lenguages ) para recuperar y modificar los datos almancenados. AQL es principalmente un lenaguaje declarativo, el que especificamos que queremos obtener y no como obtenerlo.
A continuació haremos un simple CRUD para familiarizarnos con AQL

In [27]:
doc2=coleccionEstudiantes.createDocument()
doc2={'_key':'key0','nombre':'Jose','edad':23,'tareaspromedio':[[10,7,8],[8,5,8]],'examenes':{'mate':2,'fisica':4}}
bind = {"doc": doc2} #usar un especie de alias para no acceder directamente a doc3 para evitar cualquier problema de inyeccionsql
aql = "INSERT @doc INTO Estudiantes LET ndocumento = NEW RETURN ndocumento"
queryResult = db.AQLQuery(aql, bindVars=bind)

In [28]:
doc3=coleccionEstudiantes.createDocument()
doc3={'_key':'Andres8','nombre':'Eljose','edad':23,'tareaspromedio':["HOLA",2,3.3], 'examenes':{'mate':2,'fisica':4}}
bind = {"doc": doc3}
aql = "INSERT @doc INTO Estudiantes RETURN NEW"
queryResult = db.AQLQuery(aql, bindVars=bind)
queryResult

In [29]:
doc4=coleccionEstudiantes.createDocument()
doc4={'_key':'prueba0','nombre':'prueba1','edad':23,'tareaspromedio':["HOLA",2,3.3], 'examenes':{'mate':2,'fisica':4}}
bind = {"doc": doc4}
aql = "INSERT @doc INTO Estudiantes"
queryResult = db.AQLQuery(aql, bindVars=bind)
queryResult

#### Insert de JSON file 

In [41]:
db.createCollection(name="Delitos")

ArangoDB collection name: Delitos, id: 191979, type: document, status: loaded

In [42]:
db['Delitos']

ArangoDB collection name: Delitos, id: 191979, type: document, status: loaded

In [43]:
with open('denuncias-victimas.json', 'r') as f:
    data = js.load(f)

In [45]:
bind = {"dat": data}
aql = "FOR d IN @dat INSERT d INTO Delitos"
queryResult = db.AQLQuery(aql, bindVars=bind)

In [59]:
queryResult=db.AQLQuery( "RETURN LENGTH(Delitos)")
queryResult.response

{'result': [67979],
 'hasMore': False,
 'cached': False,
 'extra': {'stats': {'writesExecuted': 0,
   'writesIgnored': 0,
   'scannedFull': 0,
   'scannedIndex': 0,
   'filtered': 0,
   'httpRequests': 0,
   'executionTime': 0.0009016990661621094,
   'peakMemoryUsage': 2000},
  'warnings': []},
 'error': False,
 'code': 201}

### Reading Documents

In [46]:
aql="""FOR c IN Delitos
LIMIT 1
RETURN c"""
qdelitos = db.AQLQuery(aql)
qdelitos.response

{'result': [{'_key': '192015',
   '_id': 'Delitos/192015',
   '_rev': '_ZNTnYWa---',
   'datasetid': 'denuncias-victimas-pgj',
   'recordid': 'a79dcb8d9eacf5fb9495843d1c4bbde264b57508',
   'fields': {'categoria': 'OTROS ROBOS SIN VIOLENCIA',
    'tipopersona': 'FISICA',
    'calidadjuridica': 'VICTIMA Y DENUNCIANTE',
    'edad': '43',
    'mes_ao': '2019-01-01',
    'geopoint': [19.3805959399, -99.16798408],
    'lon': '-99.1679840800051',
    'ao': '2019',
    'sexo': 'Masculino',
    'lat': '19.3805959398694',
    'idcarpeta': 8339890,
    'mes': 'Enero',
    'fechahecho': '24/01/2019 07:15',
    'delito': 'ROBO DE ACCESORIOS DE AUTO'},
   'geometry': {'type': 'Point', 'coordinates': [-99.16798408, 19.3805959399]},
   'record_timestamp': '2019-02-11T16:16:24-06:00'}],
 'hasMore': False,
 'cached': False,
 'extra': {'stats': {'writesExecuted': 0,
   'writesIgnored': 0,
   'scannedFull': 1,
   'scannedIndex': 0,
   'filtered': 0,
   'httpRequests': 0,
   'executionTime': 0.000365734100

### Actulizar un registro

In [47]:
aql='''UPDATE "192015" WITH {fields:{categoria: "LALALALALALALAL"}} IN Delitos'''
result=db.AQLQuery(aql)
result.response

{'result': [],
 'hasMore': False,
 'cached': False,
 'extra': {'stats': {'writesExecuted': 1,
   'writesIgnored': 0,
   'scannedFull': 0,
   'scannedIndex': 0,
   'filtered': 0,
   'httpRequests': 0,
   'executionTime': 0.0005435943603515625,
   'peakMemoryUsage': 34184},
  'warnings': []},
 'error': False,
 'code': 201}

In [53]:
def report(document):
    print(document['fields'] )
    print(document['geometry'] )
delito=db['Delitos']    
documento=delito['192015']
report(documento)
documento
##Vemos que si se hubo un update

<store: {'ao': '2019', 'calidadjuridica': 'VICTIMA Y DENUNCIANTE', 'categoria': 'LALALALALALALAL', 'delito': 'ROBO DE ACCESORIOS DE AUTO', 'edad': '43', 'fechahecho': '24/01/2019 07:15', 'geopoint': [19.3805959399, -99.16798408], 'idcarpeta': 8339890, 'lat': '19.3805959398694', 'lon': '-99.1679840800051', 'mes': 'Enero', 'mes_ao': '2019-01-01', 'sexo': 'Masculino', 'tipopersona': 'FISICA'}>
<store: {'type': 'Point', 'coordinates': [-99.16798408, 19.3805959399]}>


ArangoDoc '_id: Delitos/192015, _key: 192015, _rev: _ZNTseR---_': <store: {'datasetid': 'denuncias-victimas-pgj', 'recordid': 'a79dcb8d9eacf5fb9495843d1c4bbde264b57508', 'fields': <store: {'ao': '2019', 'calidadjuridica': 'VICTIMA Y DENUNCIANTE', 'categoria': 'LALALALALALALAL', 'delito': 'ROBO DE ACCESORIOS DE AUTO', 'edad': '43', 'fechahecho': '24/01/2019 07:15', 'geopoint': [19.3805959399, -99.16798408], 'idcarpeta': 8339890, 'lat': '19.3805959398694', 'lon': '-99.1679840800051', 'mes': 'Enero', 'mes_ao': '2019-01-01', 'sexo': 'Masculino', 'tipopersona': 'FISICA'}>, 'geometry': <store: {'type': 'Point', 'coordinates': [-99.16798408, 19.3805959399]}>, 'record_timestamp': '2019-02-11T16:16:24-06:00'}>

### Eliminar registro

In [31]:
aql="REMOVE '192015' IN Delitos" 
result=db.AQLQuery(aql)
result.response

{'result': [],
 'hasMore': False,
 'cached': False,
 'extra': {'stats': {'writesExecuted': 1,
   'writesIgnored': 0,
   'scannedFull': 0,
   'scannedIndex': 0,
   'filtered': 0,
   'httpRequests': 0,
   'executionTime': 0.003765583038330078,
   'peakMemoryUsage': 17832},
  'warnings': []},
 'error': False,
 'code': 201}

In [ ]:
#se recomiennnda no correr o aplicar mas filters
bind = {"@collection": "Delitos"}
aql = """
FOR x IN @@collection
FILTER x.fields.tipopersona == 'FISICA'
REMOVE x IN @@collection
LET removed = OLD RETURN removed
"""
queryResult = db.AQLQuery(aql, bindVars=bind)
queryResult.response

### Matching documents
Para encontrar documentos con cierta condición hay una operación de Filtrado en AQL

In [54]:
bind = {"@collection": "Delitos"}
aql = """FOR c IN @@collection
    FILTER c.fields.edad == '22'
    FILTER c.fields.mes == 'Julio' 
    FILTER c.fields.delito=='VIOLACION'
    LIMIT 3
    RETURN { anio: c.fields.ao, edad: c.fields.edad }"""
queryResult = db.AQLQuery(aql, bindVars=bind)
queryResult.response

{'result': [{'anio': '2006', 'edad': '22'}],
 'hasMore': False,
 'cached': False,
 'extra': {'stats': {'writesExecuted': 0,
   'writesIgnored': 0,
   'scannedFull': 67979,
   'scannedIndex': 0,
   'filtered': 67978,
   'httpRequests': 0,
   'executionTime': 0.24913454055786133,
   'peakMemoryUsage': 988375},
  'warnings': []},
 'error': False,
 'code': 201}

In [56]:
bind = {"@collection": "Delitos"}
aql="""FOR c IN @@collection
    SORT c.fields.mes
    LIMIT 5
    RETURN c.fields.mes"""
queryResult = db.AQLQuery(aql, bindVars=bind)
queryResult.response

{'result': ['Abril', 'Abril', 'Abril', 'Abril', 'Abril'],
 'hasMore': False,
 'cached': False,
 'extra': {'stats': {'writesExecuted': 0,
   'writesIgnored': 0,
   'scannedFull': 67979,
   'scannedIndex': 0,
   'filtered': 0,
   'httpRequests': 0,
   'executionTime': 0.18418478965759277,
   'peakMemoryUsage': 408601},
  'warnings': []},
 'error': False,
 'code': 201}

In [60]:
bind = {"@collection": "Delitos"}
aql3="""FOR doc IN WITHIN(@@collection,19.324806, -99.181495, 0.5*1000)
    RETURN {
        delitos: doc.fields.delito,
        measanio: doc.fields.mes_ao   
        }"""

In [62]:
#Nota importante antes de correr el query asegurese de haber creado un geo index con el campo
# fields.geopoint
queryResult = db.AQLQuery(aql3, bindVars=bind)
queryResult.response

{'result': [{'delitos': 'ROBO DE MOTOCICLETA SIN VIOLENCIA',
   'measanio': '2019-03-01'},
  {'delitos': 'ROBO DE MOTOCICLETA SIN VIOLENCIA', 'measanio': '2019-03-01'},
  {'delitos': 'ROBO DE OBJETOS', 'measanio': '2019-03-01'},
  {'delitos': 'FALSIFICACION DE TITULOS AL PORTADOR Y DOCUMENTOS DE CREDITO PUBLICO',
   'measanio': '01-02-19'},
  {'delitos': 'DAÑO EN PROPIEDAD AJENA CULPOSA POR TRÁNSITO VEHICULAR A AUTOMOVIL',
   'measanio': '2019-03-01'},
  {'delitos': 'ROBO DE VEHICULO DE PEDALES', 'measanio': None},
  {'delitos': 'ROBO DE VEHICULO DE PEDALES', 'measanio': '2019-01-01'},
  {'delitos': 'ROBO DE OBJETOS DEL INTERIOR DE UN VEHICULO',
   'measanio': '01-02-19'},
  {'delitos': 'ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VIOLENCIA',
   'measanio': '2019-01-01'},
  {'delitos': 'ROBO DE VEHICULO DE PEDALES', 'measanio': '01-02-19'},
  {'delitos': 'DENUNCIA DE HECHOS', 'measanio': '2014-10-01'},
  {'delitos': 'ROBO DE OBJETOS', 'measanio': '2019-03-01'},
  {'delitos': 'ROBO DE V